In [ ]:
import dotenv
import os
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

In [2]:
dotenv_path = dotenv.find_dotenv()

In [3]:
dotenv.load_dotenv(dotenv_path)

True

In [4]:
username = os.getenv("username2")
password = os.getenv("password")
host = os.getenv("host")
dbname = os.getenv("dbname")

In [5]:
connection_str = f"mysql+pymysql://{username}:{password}@{host}/{dbname}"

In [6]:
engine = create_engine(connection_str)

In [7]:
query = "SELECT * FROM dimproduct"

In [8]:
sqldf = pd.read_sql_query(query, engine)

In [12]:
sqldf["rounded_dealer_price"] = round(sqldf["DealerPrice"],2)

In [14]:
sqldf.loc[:,["DealerPrice", "rounded_dealer_price"]]

,DealerPrice,rounded_dealer_price
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
601,60.744,60.74
602,72.894,72.89
603,323.994,323.99
604,323.994,323.99


In [15]:
sqldf["DealerPriceClipped"] = sqldf["DealerPrice"].clip(lower=0, upper=1000)

In [19]:
sqldf["DealerPriceClipped"].min(), sqldf["DealerPriceClipped"].max()

(1.374, 1000.0)

In [21]:
years = 5 
a = (pd.DataFrame({"Mese": list("GFMAMGLASOND"*years),
                   "Anno": np.repeat(list(range(years)), 12),
                   "Guadagni": np.random.randint(800, 5000, 12*years)}))

In [26]:
a = a.sort_values(by="Mese", ascending=True)

In [29]:
cumulative_sum = a.groupby("Mese")["Guadagni"].cumsum()

In [31]:
a["rt"] = cumulative_sum

In [32]:
a

,Mese,Anno,Guadagni,rt
27,A,2,1800,1800
55,A,4,2365,4165
3,A,0,2444,6609
31,A,2,2378,8987
51,A,4,3457,12444
19,A,1,2156,14600
7,A,0,1390,15990
15,A,1,2130,18120
43,A,3,4040,22160
39,A,3,1168,23328


In [34]:
dimcustomer = pd.read_sql_query("SELECT * FROM dimcustomer", engine)

In [37]:
dimcustomer["FirstName"] = dimcustomer["FirstName"].str.lower()
dimcustomer["LastName"] = dimcustomer["LastName"].str.upper()

In [44]:
dimcustomer["StrippedPhone"] = dimcustomer["Phone"].str.split()

In [45]:
dimcustomer[["Phone","StrippedPhone"]]

,Phone,StrippedPhone
0,1 (11) 500 555-0162,"[1, (11), 500, 555-0162]"
1,1 (11) 500 555-0110,"[1, (11), 500, 555-0110]"
2,1 (11) 500 555-0184,"[1, (11), 500, 555-0184]"
3,1 (11) 500 555-0162,"[1, (11), 500, 555-0162]"
4,1 (11) 500 555-0131,"[1, (11), 500, 555-0131]"
...,...,...
18479,1 (11) 500 555-0136,"[1, (11), 500, 555-0136]"
18480,1 (11) 500 555-0146,"[1, (11), 500, 555-0146]"
18481,1 (11) 500 555-0144,"[1, (11), 500, 555-0144]"
18482,1 (11) 500 555-0137,"[1, (11), 500, 555-0137]"


In [50]:
dimcustomer.loc[dimcustomer["EmailAddress"].str.contains("21"), ["EmailAddress"]]

,EmailAddress
64,chase21@adventure-works.com
118,alvin21@adventure-works.com
130,caroline21@adventure-works.com
163,gabriel21@adventure-works.com
203,luis21@adventure-works.com
...,...
18337,hector21@adventure-works.com
18365,cheryl21@adventure-works.com
18435,stanley21@adventure-works.com
18440,rafael21@adventure-works.com


In [56]:
mask = (dimcustomer["EmailAddress"].str.contains("20")) | (dimcustomer["EmailAddress"].str.contains("10"))
dimcustomer.loc[mask, ["EmailAddress"]]


,EmailAddress
1,eugene10@adventure-works.com
10,jacquelyn20@adventure-works.com
22,ethan20@adventure-works.com
30,bethany10@adventure-works.com
32,denise10@adventure-works.com
...,...
18395,lydia10@adventure-works.com
18397,walter10@adventure-works.com
18415,oscar20@adventure-works.com
18418,lindsey10@adventure-works.com


In [65]:
dimcustomer["email_len"] = dimcustomer["EmailAddress"].apply(lambda x: len(x))
dimcustomer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18484 entries, 0 to 18483
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CustomerKey           18484 non-null  int64  
 1   GeographyKey          18484 non-null  int64  
 2   CustomerAlternateKey  18484 non-null  object 
 3   Title                 101 non-null    object 
 4   FirstName             18484 non-null  object 
 5   MiddleName            10654 non-null  object 
 6   LastName              18484 non-null  object 
 7   NameStyle             18484 non-null  int64  
 8   BirthDate             18484 non-null  object 
 9   MaritalStatus         18484 non-null  object 
 10  Suffix                3 non-null      object 
 11  Gender                18484 non-null  object 
 12  EmailAddress          18484 non-null  object 
 13  YearlyIncome          18484 non-null  float64
 14  TotalChildren         18484 non-null  int64  
 15  NumberChildrenAtHom

In [66]:
dimcustomer[["EmailAddress","email_len"]].sort_values(by="email_len", ascending=False).head(10)

,EmailAddress,email_len
9242,christopher11@adventure-works.com,33
2655,christopher23@adventure-works.com,33
14964,christopher26@adventure-works.com,33
3512,christopher27@adventure-works.com,33
17017,christopher15@adventure-works.com,33
13417,christopher14@adventure-works.com,33
10233,christopher24@adventure-works.com,33
16661,christopher17@adventure-works.com,33
1767,christopher18@adventure-works.com,33
8768,christopher12@adventure-works.com,33


In [68]:
dimcustomer[["EmailAddress","email_len"]].sort_values(by="email_len", ascending=False).tail(10)

,EmailAddress,email_len
13515,ian4@adventure-works.com,24
10028,joe4@adventure-works.com,24
13583,roy2@adventure-works.com,24
4716,roy8@adventure-works.com,24
4687,jay6@adventure-works.com,24
12220,roy5@adventure-works.com,24
12168,mya1@adventure-works.com,24
7742,don6@adventure-works.com,24
15299,y0@adventure-works.com,22
3974,k0@adventure-works.com,22


In [69]:
dimcustomer["EmailAddress"] = dimcustomer["EmailAddress"].str.replace("adventure-works.com", "aw-db.com")

In [70]:
dimcustomer["EmailAddress"]

0             jon24@aw-db.com
1          eugene10@aw-db.com
2           ruben35@aw-db.com
3         christy12@aw-db.com
4        elizabeth5@aw-db.com
                 ...         
18479        tommy2@aw-db.com
18480        nina21@aw-db.com
18481         ivan0@aw-db.com
18482      clayton0@aw-db.com
18483        jésus9@aw-db.com
Name: EmailAddress, Length: 18484, dtype: object

In [73]:
dimcustomer.loc[dimcustomer["AddressLine1"].str.contains("Street"), "AddressLine1"]

7              942 Brook Street
12            4785 Scott Street
32           626 Bentley Street
63           4927 Virgil Street
125          8481 Zartop Street
                  ...          
18273        2155 Zartop Street
18292         3931 Scott Street
18323    134 Peachwillow Street
18333       1023 Hawkins Street
18351          8618 Rose Street
Name: AddressLine1, Length: 512, dtype: object

In [124]:
facebook = pd.read_csv("C://Users//bulan//OneDrive//Desktop//21032024/FB_data.csv", sep=",")

In [125]:
facebook

,Date,Open,High,Low,Close,Adj Close,Volume
0,6/20/2019,190.949997,191.160004,187.639999,189.529999,189.529999,14635700
1,6/21/2019,188.750000,192.000000,188.750000,191.139999,191.139999,22751200
2,6/24/2019,192.419998,193.979996,191.570007,192.600006,192.600006,15509000
3,6/25/2019,192.880005,193.139999,188.130005,188.839996,188.839996,16750300
4,6/26/2019,189.539993,190.759995,187.309998,187.660004,187.660004,12808600
...,...,...,...,...,...,...,...
248,6/15/2020,225.089996,233.770004,224.800003,232.500000,232.500000,15340300
249,6/16/2020,237.139999,238.460007,233.000000,235.649994,235.649994,15236700
250,6/17/2020,235.000000,237.589996,231.729996,235.529999,235.529999,19552800
251,6/18/2020,234.990005,236.139999,232.149994,235.940002,235.940002,15782500


In [126]:
facebook['Date'] = pd.to_datetime(facebook['Date'], format='%m/%d/%Y')

In [127]:
facebook.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       253 non-null    datetime64[ns]
 1   Open       253 non-null    float64       
 2   High       253 non-null    float64       
 3   Low        253 non-null    float64       
 4   Close      253 non-null    float64       
 5   Adj Close  253 non-null    float64       
 6   Volume     253 non-null    int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 14.0 KB


In [128]:
facebook['timestamp'] = facebook['Date'].astype('int64') // 10**9

In [129]:
facebook["Date"].dt.strftime("%Y-%m-%d")

0      2019-06-20
1      2019-06-21
2      2019-06-24
3      2019-06-25
4      2019-06-26
          ...    
248    2020-06-15
249    2020-06-16
250    2020-06-17
251    2020-06-18
252    2020-06-19
Name: Date, Length: 253, dtype: object

In [130]:
facebook.set_index("Date", inplace=True)

In [131]:
facebook.loc["2020"]

,Open,High,Low,Close,Adj Close,Volume,timestamp
Date,,,,,,,
2020-01-02,206.750000,209.789993,206.270004,209.779999,209.779999,12077100,1577923200
2020-01-03,207.210007,210.399994,206.949997,208.669998,208.669998,11188400,1578009600
2020-01-06,206.699997,212.779999,206.520004,212.600006,212.600006,17058900,1578268800
2020-01-07,212.820007,214.580002,211.750000,213.059998,213.059998,14912400,1578355200
2020-01-08,213.000000,216.240005,212.610001,215.220001,215.220001,13475000,1578441600
...,...,...,...,...,...,...,...
2020-06-15,225.089996,233.770004,224.800003,232.500000,232.500000,15340300,1592179200
2020-06-16,237.139999,238.460007,233.000000,235.649994,235.649994,15236700,1592265600
2020-06-17,235.000000,237.589996,231.729996,235.529999,235.529999,19552800,1592352000


In [132]:
facebook.loc["2020-06"]

,Open,High,Low,Close,Adj Close,Volume,timestamp
Date,,,,,,,
2020-06-01,224.589996,232.440002,223.500000,231.910004,231.910004,18223800,1590969600
2020-06-02,230.940002,233.000000,226.559998,232.720001,232.720001,20919000,1591056000
2020-06-03,232.110001,232.649994,228.529999,230.160004,230.160004,15380300,1591142400
2020-06-04,229.559998,231.630005,224.610001,226.289993,226.289993,17041500,1591228800
2020-06-05,226.710007,231.350006,225.309998,230.770004,230.770004,16750400,1591315200
2020-06-08,229.029999,231.550003,227.410004,231.399994,231.399994,15466500,1591574400
2020-06-09,231.520004,239.770004,230.410004,238.669998,238.669998,27462900,1591660800
2020-06-10,240.960007,241.210007,235.279999,236.729996,236.729996,20720700,1591747200
2020-06-11,229.940002,232.889999,223.550003,224.429993,224.429993,26708200,1591833600


In [133]:
facebook.reset_index(inplace=True)

In [134]:
facebook

,Date,Open,High,Low,Close,Adj Close,Volume,timestamp
0,2019-06-20,190.949997,191.160004,187.639999,189.529999,189.529999,14635700,1560988800
1,2019-06-21,188.750000,192.000000,188.750000,191.139999,191.139999,22751200,1561075200
2,2019-06-24,192.419998,193.979996,191.570007,192.600006,192.600006,15509000,1561334400
3,2019-06-25,192.880005,193.139999,188.130005,188.839996,188.839996,16750300,1561420800
4,2019-06-26,189.539993,190.759995,187.309998,187.660004,187.660004,12808600,1561507200
...,...,...,...,...,...,...,...,...
248,2020-06-15,225.089996,233.770004,224.800003,232.500000,232.500000,15340300,1592179200
249,2020-06-16,237.139999,238.460007,233.000000,235.649994,235.649994,15236700,1592265600
250,2020-06-17,235.000000,237.589996,231.729996,235.529999,235.529999,19552800,1592352000
251,2020-06-18,234.990005,236.139999,232.149994,235.940002,235.940002,15782500,1592438400


In [138]:
facebook["weekday"] = facebook["Date"].dt.dayofweek + 1

In [143]:
facebook

,Date,Open,High,Low,Close,Adj Close,Volume,timestamp,weekday
0,2019-06-20,190.949997,191.160004,187.639999,189.529999,189.529999,14635700,1560988800,4
1,2019-06-21,188.750000,192.000000,188.750000,191.139999,191.139999,22751200,1561075200,5
2,2019-06-24,192.419998,193.979996,191.570007,192.600006,192.600006,15509000,1561334400,1
3,2019-06-25,192.880005,193.139999,188.130005,188.839996,188.839996,16750300,1561420800,2
4,2019-06-26,189.539993,190.759995,187.309998,187.660004,187.660004,12808600,1561507200,3
...,...,...,...,...,...,...,...,...,...
248,2020-06-15,225.089996,233.770004,224.800003,232.500000,232.500000,15340300,1592179200,1
249,2020-06-16,237.139999,238.460007,233.000000,235.649994,235.649994,15236700,1592265600,2
250,2020-06-17,235.000000,237.589996,231.729996,235.529999,235.529999,19552800,1592352000,3
251,2020-06-18,234.990005,236.139999,232.149994,235.940002,235.940002,15782500,1592438400,4


In [144]:
weekend = [4,5]
facebook.loc[facebook["weekday"].isin(weekend)]



,Date,Open,High,Low,Close,Adj Close,Volume,timestamp,weekday
0,2019-06-20,190.949997,191.160004,187.639999,189.529999,189.529999,14635700,1560988800,4
1,2019-06-21,188.750000,192.000000,188.750000,191.139999,191.139999,22751200,1561075200,5
5,2019-06-27,189.880005,198.880005,188.270004,189.500000,189.500000,11159000,1561593600,4
6,2019-06-28,190.550003,193.199997,189.940002,193.000000,193.000000,16378900,1561680000,5
10,2019-07-05,196.179993,197.070007,194.169998,196.399994,196.399994,11164100,1562284800,5
...,...,...,...,...,...,...,...,...,...
242,2020-06-05,226.710007,231.350006,225.309998,230.770004,230.770004,16750400,1591315200,5
246,2020-06-11,229.940002,232.889999,223.550003,224.429993,224.429993,26708200,1591833600,4
247,2020-06-12,229.899994,231.660004,224.500000,228.580002,228.580002,22071700,1591920000,5
251,2020-06-18,234.990005,236.139999,232.149994,235.940002,235.940002,15782500,1592438400,4


In [145]:
facebook['datetime_with_timezone'] = facebook['Date'].dt.tz_localize('Europe/Rome')

In [146]:
facebook

,Date,Open,High,Low,Close,Adj Close,Volume,timestamp,weekday,datetime_with_timezone
0,2019-06-20,190.949997,191.160004,187.639999,189.529999,189.529999,14635700,1560988800,4,2019-06-20 00:00:00+02:00
1,2019-06-21,188.750000,192.000000,188.750000,191.139999,191.139999,22751200,1561075200,5,2019-06-21 00:00:00+02:00
2,2019-06-24,192.419998,193.979996,191.570007,192.600006,192.600006,15509000,1561334400,1,2019-06-24 00:00:00+02:00
3,2019-06-25,192.880005,193.139999,188.130005,188.839996,188.839996,16750300,1561420800,2,2019-06-25 00:00:00+02:00
4,2019-06-26,189.539993,190.759995,187.309998,187.660004,187.660004,12808600,1561507200,3,2019-06-26 00:00:00+02:00
...,...,...,...,...,...,...,...,...,...,...
248,2020-06-15,225.089996,233.770004,224.800003,232.500000,232.500000,15340300,1592179200,1,2020-06-15 00:00:00+02:00
249,2020-06-16,237.139999,238.460007,233.000000,235.649994,235.649994,15236700,1592265600,2,2020-06-16 00:00:00+02:00
250,2020-06-17,235.000000,237.589996,231.729996,235.529999,235.529999,19552800,1592352000,3,2020-06-17 00:00:00+02:00
251,2020-06-18,234.990005,236.139999,232.149994,235.940002,235.940002,15782500,1592438400,4,2020-06-18 00:00:00+02:00


In [147]:
facebook['differenza_tempo'] = facebook['Date'].diff()

In [148]:
facebook

,Date,Open,High,Low,Close,Adj Close,Volume,timestamp,weekday,datetime_with_timezone,differenza_tempo
0,2019-06-20,190.949997,191.160004,187.639999,189.529999,189.529999,14635700,1560988800,4,2019-06-20 00:00:00+02:00,NaT
1,2019-06-21,188.750000,192.000000,188.750000,191.139999,191.139999,22751200,1561075200,5,2019-06-21 00:00:00+02:00,1 days
2,2019-06-24,192.419998,193.979996,191.570007,192.600006,192.600006,15509000,1561334400,1,2019-06-24 00:00:00+02:00,3 days
3,2019-06-25,192.880005,193.139999,188.130005,188.839996,188.839996,16750300,1561420800,2,2019-06-25 00:00:00+02:00,1 days
4,2019-06-26,189.539993,190.759995,187.309998,187.660004,187.660004,12808600,1561507200,3,2019-06-26 00:00:00+02:00,1 days
...,...,...,...,...,...,...,...,...,...,...,...
248,2020-06-15,225.089996,233.770004,224.800003,232.500000,232.500000,15340300,1592179200,1,2020-06-15 00:00:00+02:00,3 days
249,2020-06-16,237.139999,238.460007,233.000000,235.649994,235.649994,15236700,1592265600,2,2020-06-16 00:00:00+02:00,1 days
250,2020-06-17,235.000000,237.589996,231.729996,235.529999,235.529999,19552800,1592352000,3,2020-06-17 00:00:00+02:00,1 days
251,2020-06-18,234.990005,236.139999,232.149994,235.940002,235.940002,15782500,1592438400,4,2020-06-18 00:00:00+02:00,1 days


In [149]:
pokemon = pd.read_csv("C://Users//bulan//OneDrive//Desktop//21032024/pokemon.csv")

In [150]:
pokemon

,abilities,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,...,percentage_male,pokedex_number,sp_attack,sp_defense,speed,type1,type2,weight_kg,generation,is_legendary
0,"['Overgrow', 'Chlorophyll']",1.00,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,1,65,65,45,grass,poison,6.9,1,0
1,"['Overgrow', 'Chlorophyll']",1.00,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,2,80,80,60,grass,poison,13.0,1,0
2,"['Overgrow', 'Chlorophyll']",1.00,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,3,122,120,80,grass,poison,100.0,1,0
3,"['Blaze', 'Solar Power']",0.50,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,4,60,50,65,fire,NaN,8.5,1,0
4,"['Blaze', 'Solar Power']",0.50,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,5,80,65,80,fire,NaN,19.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,['Beast Boost'],0.25,1.0,0.5,2.0,0.5,1.0,2.0,0.5,1.0,...,NaN,797,107,101,61,steel,flying,999.9,7,1
797,['Beast Boost'],1.00,1.0,0.5,0.5,0.5,2.0,4.0,1.0,1.0,...,NaN,798,59,31,109,grass,steel,0.1,7,1
798,['Beast Boost'],2.00,0.5,2.0,0.5,4.0,2.0,0.5,1.0,0.5,...,NaN,799,97,53,43,dark,dragon,888.0,7,1
799,['Prism Armor'],2.00,2.0,1.0,1.0,1.0,0.5,1.0,1.0,2.0,...,NaN,800,127,89,79,psychic,NaN,230.0,7,1


In [160]:
len(pokemon.loc[pokemon.isna().any(axis=1)].index)

462

In [167]:
valori_nulli_per_riga = pokemon.isnull().sum(axis=1)

# Aggiunta della colonna al DataFrame
pokemon['valori_nulli_per_riga'] = valori_nulli_per_riga

0      False
1      False
2      False
3       True
4       True
       ...  
796     True
797     True
798     True
799     True
800     True
Length: 801, dtype: bool

In [182]:
df_con_valori_nulli = []
null_value = 0

directory = "C://Users//bulan//OneDrive//Desktop//21032024//"

for file in os.listdir():
    if file.endswith("csv"):
        df = pd.read_csv(os.path.join(directory, file))
        if df.isnull().any().any():
            df_con_valori_nulli.append(file)
        
print("DataFrame con valori nulli:")
print(df_con_valori_nulli)


DataFrame con valori nulli:
['pokemon.csv', 'test.csv']
